### Initialization

In [82]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload
%autoreload 2

import sys
import os
import time
from copy import deepcopy
sys.path.insert(0, './src')
# print(sys.path)

import wandb
import numpy as np
import torch
from torch.utils.data import DataLoader
# from sklearn.metrics import confusion_matrix
# from tqdm import tqdm, trange

from utils.notebook_modules import initialize, init_dataloaders, init_environment, init_wandb, \
                                   training_prep, retrain_prep, retrain_phase,  \
                                   disp_dataloader_info,disp_info_1, disp_for_excel, disp_gpu_info

from utils.util import (print_separator, print_underline, print_yaml,  print_loss, 
                        timestring, print_heading, print_dbg, get_command_line_args ) 


# from envs.sparsechem_env_dev import SparseChemEnv_Dev
# from utils.sparsechem_utils import load_sparse, load_task_weights, class_fold_counts, fold_and_transform_inputs, print_metrics_cr
# from dataloaders.chembl_dataloader_dev import ClassRegrSparseDataset_v3, ClassRegrSparseDataset, InfiniteDataLoader
from utils.util import ( makedir, print_separator, create_path, print_yaml, print_yaml2, print_loss, should, 
                         fix_random_seed, read_yaml, timestring, print_heading, print_dbg, save_to_pickle, load_from_pickle,
                         print_underline, write_config_report, display_config, get_command_line_args, is_notebook) 
### Read Configuration File### Initialization
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Retrain.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Parse Input Args & Read yaml config file

In [29]:
RETRAIN_FROM_PATH  = '../experiments/AdaSparseChem/50x6_0308_1204_plr0.01_sp0.01_sh0.01'
RETRAIN_MODEL_CKPT = 'model_train_ep_25_seed_0088'

In [6]:
input_args = " --config yamls/chembl_3task_retrain.yaml " \
             " --exp_name    0308_1204" \
             " --folder_sfx    Retrain" \
             " --exp_desc Retrain phase of 0308_1204 run - 1" \
             " --seed_idx 0 "    \
             " --batch_size 128" \
             " --lambda_sparsity  0.01"\
             " --lambda_sharing   0.01" 
### Read Configuration File
# get command line arguments
# args = get_command_line_args(input_args.split())

opt, ns = initialize(input_args)


  command line parms : 
------------------------
 config...................  yamls/chembl_3task_retrain.yaml
 exp_id...................  None
 exp_name.................  0308_1204
 folder_sfx...............  Retrain
 exp_desc.................  Retrain phase of 0308_1204 run - 1
 seed_idx.................  0
 batch_size...............  128
 backbone_lr..............  None
 task_lr..................  None
 policy_lr................  None
 decay_lr_rate............  None
 decay_lr_freq............  None
 lambda_sparsity..........  0.01
 lambda_sharing...........  0.01
 gpu_ids..................  [0]
 resume...................  False
 cpu......................  False





### Setup Dataloader and Model  

In [11]:
dldrs = init_dataloaders(opt)

environ = init_environment(ns, opt, is_train = True, policy_learning = False, display_cfg = False)

##################################################
############### CREATE DATALOADERS ###############
##################################################
##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
 device is  cuda:0
--------------------------------------------------------
* SparseChemEnv_Dev environment successfully created
-------------------------------------------------------- 



In [68]:
print(ns.__dict__.keys())
print(dldrs.__dict__.keys())

dict_keys(['args', 'config_filename', 'wandb_run', 'eval_iters'])
dict_keys(['trainset0', 'trainset1', 'trainset2', 'valset', 'testset', 'warmup_trn_loader', 'weight_trn_loader', 'policy_trn_loader', 'val_loader', 'test_loader'])


In [12]:
disp_dataloader_info(dldrs)


 trainset.y_class                                   :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset1.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset2.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 valset.y_class                                     :  [(4137, 5), (4137, 5), (4137, 5)]  
 testset.y_class                                    :  [(920, 5), (920, 5), (920, 5)]  
                                 
 size of training set 0 (warm up)                   :  13331 
 size of training set 1 (network parms)             :  13331 
 size of training set 2 (policy weights)            :  13331 
 size of validation set                             :  4137 
 size of test set                                   :  920 
                               Total                :  45050 
                                 
 lenght (# batches) in training 0 (warm up)         :  105 
 lenght (# batches) in training 1 (network p

###  Weights and Biases Initialization 

In [17]:
print(opt['exp_id'], opt['exp_name'], opt['project_name']) # , opt['exp_instance'])
# opt['exp_id'] = wandb.util.generate_id()

3qg62on3 0308_1204_Retrain AdaSparseChem


In [18]:
init_wandb(ns, opt, environment = environ)

# print(f" PROJECT NAME: {ns.wandb_run.project}\n"
#       f" RUN ID      : {ns.wandb_run.id} \n"
#       f" RUN NAME    : {ns.wandb_run.name}") 

3qg62on3 0308_1204_Retrain AdaSparseChem


 PROJECT NAME: AdaSparseChem
 RUN ID      : 3qg62on3 
 RUN NAME    : 0308_1204_Retrain
 PROJECT NAME: AdaSparseChem
 RUN ID      : 3qg62on3 
 RUN NAME    : 0308_1204_Retrain


In [19]:
# wandb.config = opt.copy()
# wandb.watch(environ.networks['mtl-net'], log='all', log_freq=1000)     ###  Weights and Biases Initialization 
# run.finish()


### Load Model Snapshot and  Saved Policy (if present)

Load the following files 
    -  Desired checkpoint 
    -  Policy file 

In [90]:
current_iter, current_epoch = 0, 0
opt['retrain_from_pl'], environ.opt['retrain_from_pl'] = True, True
policy_label = 'policy_ep_%s_seed_%04d' % (25, opt['random_seed'])
# policy_label = 'policy_ep_%s_seed_%04d' % (opt['train']['policy_iter'], opt['random_seed'])

print(f" Policy Label   : {policy_label}")
# print(f" Retrain Resume : {opt['train']['retrain_resume']}, {environ.opt['train']['retrain_resume']}")
# print(f" Retrain from PL: {opt['retrain_from_pl']}, {environ.opt['retrain_from_pl']}")
print(f" Hard sampling  : {opt['train']['hard_sampling']}")
print(opt['train']['which_iter'])

 Policy Label   : policy_ep_25_seed_0088
 Hard sampling  : False
warmup


In [91]:
environ.define_optimizer(policy_learning=True)
environ.define_scheduler(policy_learning=True)


# ********************************************************************
# **************** Initialize the environment ************************
# ********************************************************************    
if opt['train']['retrain_resume']:
    print_separator('CONTINUE RETRAIN PHASE')    
    data = environ.load_checkpoint(opt['train']['which_iter'], verbose = True)
    if opt['policy_model'] == 'task-specific':
        environ.load_policy(policy_label)
else:
    print_separator('START RETRAIN PHASE')       
    if opt['policy_model'] == 'task-specific':
            print_underline(f"I. Deep copy environ.get_current_state(0)", verbose= True)
            init_state = deepcopy(environ.get_current_state(0))

            print_underline(f"\n II. environ.check_exist_policy() Check if {policy_label} exists", verbose = True)
            
            if environ.check_exist_policy(policy_label):
                print_underline(f"\n A. load existing POLICY: {policy_label}", verbose = True)
                environ.load_policy(policy_label)
            else:         
                print_underline(f"\n 1. load checkpoint : {RETRAIN_FROM_PATH} - {RETRAIN_MODEL_CKPT}", verbose = True)
                loaded_iter, loaded_epoch = environ.load_checkpoint(RETRAIN_MODEL_CKPT, path=RETRAIN_FROM_PATH, verbose = True)
             
                
                print_underline("\n 2. get_policy_logits from network['mtl-net']", verbose = True)
                dists = environ.get_policy_logits()
                overall_logits = np.concatenate(dists, axis=-1)
                print(f' {type(overall_logits)}  - {overall_logits.shape}\n {overall_logits} \n')

                print_underline(f"\n 3. environ.get_policy_prob(): Get logits from networks['mtl-net'], apply softmax, and print", verbose = True)
                dists = environ.get_policy_prob()
                overall_dist = np.concatenate(dists, axis=-1)
                print(f' {type(overall_dist)}  - {overall_dist.shape}\n {overall_dist} \n')

                print_underline(f"\n 4. environ.sample_policy() : call test_sample_policy(hard_sampling = "
                                f"{opt['train']['hard_sampling']}) and save as environ.policy<i> attributes", verbose = True)
                environ.sample_policy(opt['train']['hard_sampling'])
                print(f"logits: \n {np.concatenate(environ.get_current_logits(), axis=-1)}")
                print(f"policies: \n {np.concatenate(environ.get_current_policy(), axis=-1)}")
                
                print_underline(f'\n 5. environ.save_policy() : save environ.policy<i> attributes to file {policy_label}', verbose = True)
                environ.save_policy(policy_label)

            if opt['retrain_from_pl']:
                print_underline(f"\n 6. load checkpoint {opt['train']['policy_iter']}", verbose = True)            
#                 environ.load_checkpoint(opt['train']['policy_iter'])
                loaded_iter, loaded_epoch = environ.load_checkpoint(RETRAIN_MODEL_CKPT, path=RETRAIN_FROM_PATH, verbose = True)
            else:
                print_underline(f"\n 7. load snapshot from init_state ( get_current_state(0) )", verbose = True)                  
                loaded_iter, loaded_epoch = environ.load_snapshot(init_state)


print_underline(f"\n get_current_policy(): Get environ.policy<i> attributes and return as list", verbose = True)          
policys = environ.get_current_policy()
overall_policy = np.concatenate(policys, axis=-1)
print(overall_policy)

##################################################
############## START RETRAIN PHASE ###############
##################################################

 I. Deep copy environ.get_current_state(0)
-------------------------------------------
 sparsechem_env-dev get_current_state()   0    unknown

 
 II. environ.check_exist_policy() Check if policy_ep_25_seed_0088 exists
---------------------------------------------------------------------------

 
 A. load existing POLICY: policy_ep_25_seed_0088
---------------------------------------------------

 
 6. load checkpoint best
---------------------------
=> loading snapshot from ../experiments/AdaSparseChem/50x6_0308_1204_plr0.01_sp0.01_sh0.01/model_train_ep_25_seed_0088_model.pth.tar
   Loading to GPU cuda:0
  networks -  network:  mtl-net
  load snapshot - network:  mtl-net
    network mtl-net - item task1_logits
    network mtl-net - item task2_logits
    network mtl-net - item task3_logits
    network mtl-net - item backbone.Input_line

In [92]:
print(loaded_iter, loaded_epoch)
# current_epoch = 170

3675 25


In [93]:
# print('\n sample policy()')
# print('-----------------')
# environ.sample_policy(hard_sampling = opt['train'][R'hard_sampling'], verbose = True)
# environ.sample_policy(hard_sampling = False, verbose = True)
# print(environ.networks['mtl-net'].policys)
# print(environ.networks['mtl-net'].policy1)

# environ.display_trained_logits(0)
# environ.display_trained_policy(0)

# print_underline(f"get_current_policy()",verbose =True) 
# policys = environ.get_current_policy()
# print( np.concatenate(policys, axis=-1))

In [94]:
# print(environ.networks['mtl-net'].num_layers, environ.networks['mtl-net'].skip_layer)
# environ.networks['mtl-net'].policys
# 
# environ.display_test_sample_policy(0, hard_sampling = False)
# environ.display_test_sample_policy(0, hard_sampling = True)
# 
# environ.display_train_sample_policy(0, hard_sampling = False)
# environ.display_train_sample_policy(0, hard_sampling = True)

# print(f"\n 8. get_current_policy")          
# policys = environ.get_current_policy()
# overall_policy = np.concatenate(policys, axis=-1)
# print(overall_policy)

In [95]:
print(environ.optimizers['alphas'])
print(environ.optimizers['weights'])

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.01
    lr: 0.01
    weight_decay: 0.0005
)
SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.001
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001

Parameter Group 1
    dampening: 0
    initial_lr: 0.001
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)


### Training Preparation

In [96]:
retrain_prep(ns, opt, environ, dldrs, phase = 'update_w', epoch = loaded_epoch, iter = loaded_iter )

 set eval_iters to length of val loader  : 33
opt['train']['retrain_total_iters']:   25000


In [97]:
print(ns.__dict__.keys())
print(dldrs.__dict__.keys())

dict_keys(['args', 'config_filename', 'wandb_run', 'eval_iters', 'stop_iter_w', 'flag', 'best_results', 'best_metrics', 'best_value', 'best_iter', 'current_epoch', 'current_iter', 'stop_epoch_training', 'training_epochs'])
dict_keys(['trainset0', 'trainset1', 'trainset2', 'valset', 'testset', 'warmup_trn_loader', 'weight_trn_loader', 'policy_trn_loader', 'val_loader', 'test_loader'])


In [99]:
retrain_phase(ns, opt, environ, dldrs, epochs = 10, display_policy = False, verbose = False)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed: 35   # of epochs to do:  10 -  epochs 36 to 45
 stop_iter_w         : 105
 policy_lr           : 0.01
 lambda_sparsity     : 0.01
 lambda_sharing      : 0.01
------------------------------------------------------------------------------------------------------------------------ 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   36 |   8.50e-04   8.50e-04   1.00e-02  3.347e+00 |    2.6231   2.1375e+00   3.2885e-02    4.7935 |   0.60225   0.80157   0.80549   0.80144 |    9.0365   2.1375e+00   3.2885e-02    11.2069 |  21.1 |
   37 |   8.50e-04   8.50e-04   1.00e-02  3.347e+00 |    2.7483   2.1375e+00   3.2885e-02    4.9187 |   0.60355   0.80160   0.80570   0.80147 |    9.0616   2.1375e+00   3.28

In [52]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']:.3e}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']:.3e}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")

print()
print( f" current_iters               : {current_iter}")  
print( f" current_epochs              : {current_epoch}") 
print( f" train_total_epochs          : {train_total_epochs}") 
print( f" stop_epoch_training         : {stop_epoch_training}")

 Backbone Learning Rate      : 1.000e-03
 Tasks    Learning Rate      : 1.000e-03
 Policy   Learning Rate      : 0.01


 current_iters               : 35740
 current_epochs              : 180
 train_total_epochs          : 10
 stop_epoch_training         : 180


In [ ]:
# environ.opt['train']['backbone_lr'] = 0.0005
# environ.opt['train']['task_lr']     = 0.0005


#  2.72e-04   2.72e-04 

In [48]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")

print()
print( f" current_iters               : {current_iter}")  
print( f" current_epochs              : {current_epoch}") 
print( f" train_total_epochs          : {train_total_epochs}") 
print( f" stop_epoch_training         : {stop_epoch_training}")

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.01


 current_iters               : 35740
 current_epochs              : 180
 train_total_epochs          : 10
 stop_epoch_training         : 180


In [25]:
run.finish()

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_time,▃▁▁▅▁▁▅█▅▄▃▅▄▂▄▃▂▄▂▄
epoch,190
train_time,23.36417


In [57]:
val_metrics['aggregated']



{'roc_auc_score': 0.7977660069319655,
 'auc_pr': 0.8014618586453754,
 'avg_prec_score': 0.8015433942620374,
 'f1_max': 0.7484090681503753,
 'p_f1_max': 0.12281983842452368,
 'kappa': 0.43709999925532184,
 'kappa_max': 0.44975542850744377,
 'p_kappa_max': 0.5509484738111496,
 'bceloss': 0.9510966817537944,
 'sc_loss': 0.3957416854852666,
 'logloss': 0.0002082394310819206}

In [40]:
val_metrics['aggregated']



{'roc_auc_score': 0.797781100801785,
 'auc_pr': 0.8018675064848862,
 'avg_prec_score': 0.8019504284471113,
 'f1_max': 0.7494570643715834,
 'p_f1_max': 0.08885766727228958,
 'kappa': 0.44084984851153447,
 'kappa_max': 0.44978186934986036,
 'p_kappa_max': 0.5494123538335165,
 'bceloss': 1.115734883149465,
 'sc_loss': 0.46436059796888013,
 'logloss': 0.00024434672991127215}

In [53]:
# load_from_pickle(environ.opt['paths']['checkpoint_dir'], metrics_label)

In [45]:
#     environ.save_checkpoint('retrain%03d_policyIter%s_latest' % (exp_id, opt['train']['policy_iter']), current_iter)
    # end validation
    
#     # validation             
#     if should(current_iter, opt['train']['val_freq']):
#         environ.eval()
#         num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
#         val_metrics = eval_fix_policy(environ, val_loader, opt['tasks'], num_seg_class)
#         environ.print_loss(current_iter, start_time, val_metrics)
#         environ.save_checkpoint('retrain%03d_policyIter%s_latest' % (exp_id, opt['train']['policy_iter']), current_iter)
#         environ.train()
 
##
## Check for best metrics and checkpoint if justified
##
#         best_value, best_iter, best_metrics = check_for_best_metrics(best_value, current_iter, refer_metrics, val_metrics, opt)

    # end validation 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 28)

### Functions

In [ ]:
def main():
    # ********************************************************************
    # ****************** create folders and print options ****************
    # ********************************************************************
    print_separator('READ YAML')
    opt, gpu_ids, exp_ids = read_yaml()
    # fix_random_seed(opt["seed"])
    create_path(opt)
    # print yaml on the screen
    lines = print_yaml(opt)
    for line in lines: print(line)
    # print to file
    with open(os.path.join(opt['paths']['log_dir'], opt['exp_name'], 'opt.txt'), 'w+') as f:
        f.writelines(lines)

    best_results = {}
    for exp_id in exp_ids:
        fix_random_seed(opt["seed"][exp_id])
        # fix_random_seed(48)
        _, policy = train(exp_id, opt, gpu_ids)


if __name__ == "__main__":
    main()

In [ ]:
def init_records(tasks, num_seg_cls):
    records = {}
    if 'seg' in tasks:
        assert (num_seg_cls != -1)
    records['seg'] = {'mIoUs'    : [], 
                      'pixelAccs': [],  
                      'errs'     : [], 
                      'conf_mat' : np.zeros((num_seg_cls, num_seg_cls)),
                      'labels'   : np.arange(num_seg_cls), 
                      'gts': [], 
                      'preds': []}
    if 'sn' in tasks:
        records['sn'] = {'cos_similaritys': []}
    if 'depth' in tasks:
        records['depth'] = {'abs_errs': [], 
                            'rel_errs': [], 
                            'sq_rel_errs': [], 
                            'ratios'  : [], 
                            'rms'     : [], 
                            'rms_log' : []}
    if 'keypoint' in tasks:
        records['keypoint'] = {'errs': []}
    if 'edge' in tasks:
        records['edge'] = {'errs': []}
    return records

def populate_records(recs, mtrs,tsks ):
    if 'seg' in tasks:
        new_mat = confusion_matrix(metrics['seg']['gt'], metrics['seg']['pred'], records['seg']['labels'])
        assert (records['seg']['conf_mat'].shape == new_mat.shape)
        records['seg']['conf_mat'] += new_mat
        records['seg']['pixelAccs'].append(metrics['seg']['pixelAcc'])
        records['seg']['errs'].append(metrics['seg']['err'])
        records['seg']['gts'].append(metrics['seg']['gt'])
        records['seg']['preds'].append(metrics['seg']['pred'])

    if 'sn' in tasks:
        records['sn']['cos_similaritys'].append(metrics['sn']['cos_similarity'])

    if 'depth' in tasks:
        records['depth']['abs_errs'].append(metrics['depth']['abs_err'])
        records['depth']['rel_errs'].append(metrics['depth']['rel_err'])
        records['depth']['sq_rel_errs'].append(metrics['depth']['sq_rel_err'])
        records['depth']['ratios'].append(metrics['depth']['ratio'])
        records['depth']['rms'].append(metrics['depth']['rms'])
        records['depth']['rms_log'].append(metrics['depth']['rms_log'])

    if 'keypoint' in tasks:
        records['keypoint']['errs'].append(metrics['keypoint']['err'])

    if 'edge' in tasks:
        records['edge']['errs'].append(metrics['edge']['err'])

    return recs

def populate_val_metrics(records, tasks, num_seg_cls):
    val_metrics = {}
    
    if 'seg' in tasks:
        val_metrics['seg'] = {}
        jaccard_perclass = []
        for i in range(num_seg_cls):
            if not records['seg']['conf_mat'][i, i] == 0:
                jaccard_perclass.append(records['seg']['conf_mat'][i, i] / (np.sum(records['seg']['conf_mat'][i, :]) +
                                                                            np.sum(records['seg']['conf_mat'][:, i]) -
                                                                            records['seg']['conf_mat'][i, i]))

        val_metrics['seg']['mIoU'] = np.sum(jaccard_perclass) / len(jaccard_perclass)

        val_metrics['seg']['Pixel Acc'] = (np.array(records['seg']['pixelAccs']) * np.array(batch_size)).sum() / sum(
            batch_size)

        val_metrics['seg']['err'] = (np.array(records['seg']['errs']) * np.array(batch_size)).sum() / sum(batch_size)

    if 'sn' in tasks:
        val_metrics['sn'] = {}
        overall_cos = np.clip(np.concatenate(records['sn']['cos_similaritys']), -1, 1)

        angles = np.arccos(overall_cos) / np.pi * 180.0
        val_metrics['sn']['cosine_similarity'] = overall_cos.mean()
        val_metrics['sn']['Angle Mean'] = np.mean(angles)
        val_metrics['sn']['Angle Median'] = np.median(angles)
        val_metrics['sn']['Angle 11.25'] = np.mean(np.less_equal(angles, 11.25)) * 100
        val_metrics['sn']['Angle 22.5'] = np.mean(np.less_equal(angles, 22.5)) * 100
        val_metrics['sn']['Angle 30'] = np.mean(np.less_equal(angles, 30.0)) * 100
        val_metrics['sn']['Angle 45'] = np.mean(np.less_equal(angles, 45.0)) * 100

    if 'depth' in tasks:
        val_metrics['depth'] = {}
        records['depth']['abs_errs'] = np.stack(records['depth']['abs_errs'], axis=0)
        records['depth']['rel_errs'] = np.stack(records['depth']['rel_errs'], axis=0)
        records['depth']['ratios'] = np.concatenate(records['depth']['ratios'], axis=0)

        val_metrics['depth']['abs_err'] = (records['depth']['abs_errs'] * np.array(batch_size)).sum() / sum(batch_size)
        val_metrics['depth']['rel_err'] = (records['depth']['rel_errs'] * np.array(batch_size)).sum() / sum(batch_size)
       
        val_metrics['depth']['sigma_1.25'] = np.mean(np.less_equal(records['depth']['ratios'], 1.25)) * 100
        val_metrics['depth']['sigma_1.25^2'] = np.mean(np.less_equal(records['depth']['ratios'], 1.25 ** 2)) * 100
        val_metrics['depth']['sigma_1.25^3'] = np.mean(np.less_equal(records['depth']['ratios'], 1.25 ** 3)) * 100

    if 'keypoint' in tasks:
        val_metrics['keypoint'] = {}
        val_metrics['keypoint']['err'] = (np.array(records['keypoint']['errs']) * np.array(batch_size)).sum() / sum(
            batch_size)

    if 'edge' in tasks:
        val_metrics['edge'] = {}
        val_metrics['edge']['err'] = (np.array(records['edge']['errs']) * np.array(batch_size)).sum() / sum(batch_size)

    return val_metrics


def get_reference_metrics(opt):
    if opt['dataload']['dataset'] == 'NYU_v2':
        if len(opt['tasks_num_class']) == 2:
            refer_metrics = {'seg': {'mIoU': 0.413, 'Pixel Acc': 0.691},
                             'sn': {'Angle Mean': 15, 'Angle Median': 11.5, 'Angle 11.25': 49.2, 'Angle 22.5': 76.7,
                                    'Angle 30': 86.8}}
        elif len(opt['tasks_num_class']) == 3:
            refer_metrics = {'seg': {'mIoU': 0.275, 'Pixel Acc': 0.589},
                             'sn': {'Angle Mean': 17.5, 'Angle Median': 14.2, 'Angle 11.25': 34.9, 'Angle 22.5': 73.3,
                                    'Angle 30': 85.7},
                             'depth': {'abs_err': 0.62, 'rel_err': 0.25, 'sigma_1.25': 57.9,
                                       'sigma_1.25^2': 85.8, 'sigma_1.25^3': 95.7}}
        else:
            raise ValueError('num_class = %d is invalid' % len(opt['tasks_num_class']))

    elif opt['dataload']['dataset'] == 'CityScapes':
        num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1

        if num_seg_class == 7 and opt['dataload']['small_res']:
            refer_metrics = {'seg': {'mIoU': 0.519, 'Pixel Acc': 0.722},
                         'depth': {'abs_err': 0.017, 'rel_err': 0.33, 'sigma_1.25': 70.3,
                                   'sigma_1.25^2': 86.3, 'sigma_1.25^3': 93.3}}
        elif num_seg_class == 7 and not opt['dataload']['small_res']:
            refer_metrics = {'seg': {'mIoU': 0.644, 'Pixel Acc': 0.778},
                         'depth': {'abs_err': 0.017, 'rel_err': 0.33, 'sigma_1.25': 70.3,
                                   'sigma_1.25^2': 86.3, 'sigma_1.25^3': 93.3}}
        
        elif num_seg_class == 19 and not opt['dataload']['small_res']:
            refer_metrics = {'seg': {'mIoU': 0.402, 'Pixel Acc': 0.747},
                            'depth': {'abs_err': 0.017, 'rel_err': 0.33, 'sigma_1.25': 70.3,
                                    'sigma_1.25^2': 86.3, 'sigma_1.25^3': 93.3}}
        else:
            raise ValueError('num_seg_class = %d and small res = %d are not supported' % (num_seg_class, opt['dataload']['small_res']))
 
    elif opt['dataload']['dataset'] == 'Taskonomy':
        refer_metrics = {'seg': {'err': 0.517},
                         'sn': {'cosine_similarity': 0.716},
                         'depth': {'abs_err': 0.021},
                         'keypoint': {'err': 0.197},
                         'edge': {'err': 0.212}}
    
    else:
        raise NotImplementedError('Dataset %s is not implemented' % opt['dataload']['dataset'])
    
    return refer_metrics

def check_for_best_metrics(best_value, current_iter, refer_metrics, val_metrics, opt):
    new_value = 0
    for k in refer_metrics.keys():
        if k in val_metrics.keys():
            for kk in val_metrics[k].keys():
                if not kk in refer_metrics[k].keys():
                    continue
                if (k == 'sn' and kk in ['Angle Mean', 'Angle Median']) or (
                        k == 'depth' and not kk.startswith('sigma')) or (kk == 'err'):
                    value = refer_metrics[k][kk] / val_metrics[k][kk]
                else:
                    value = val_metrics[k][kk] / refer_metrics[k][kk]

                value = value / len(list(set(val_metrics[k].keys()) & set(refer_metrics[k].keys())))
                new_value += value

    if new_value > best_value:
        best_value = new_value
        best_metrics = val_metrics
        best_iter = current_iter
        environ.save_checkpoint('retrain%03d_policyIter%s_best' % (exp_id, opt['train']['policy_iter']), current_iter)
        
        print('new value: %.3f' % new_value)
        print('best iter: %d, best value: %.3f' % (best_iter, best_value), best_metrics)    
    return best_value, best_iter, best_metrics

In [ ]:
from utils.util import init_records, populate_records, populate_validation_metrics

def eval_fix_policy(environ, dataloader, tasks, num_seg_cls=-1, eval_iter=10):
    batch_size = []
    val_metrics = {}

    records = init_records(tasks, num_seg_cls)
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader)):
            if eval_iter != -1:
                if batch_idx > eval_iter:
                    break

            environ.set_inputs(batch)

            metrics = environ.val_fix_policy()
            
            # environ.networks['mtl-net'].task1_logits
            # mIoUs.append(mIoU)
            ##
            ##  Populate Records with results from metrics
            populate_records(records, metrics, tasks)
 
            batch_size.append(len(batch['img']))

    ##  Populate Validation Metrics 
    val_metrics = populate_validation_metrics(records, tasks, num_seg_cls, batch_size)

    return val_metrics

### Close WandB Session

In [10]:
wandb.finish()

Error in callback <function _WandbInit._resume_backend at 0x7f99f9d0e1f0> (for pre_run_cell):


AttributeError: 'NoneType' object has no attribute 'info'

Exception: Attribute _telemetry_obj_dirty is not supported on Run object.

Error in callback <function _WandbInit._pause_backend at 0x7f99f6a41310> (for post_run_cell):


AttributeError: 'NoneType' object has no attribute 'info'